In [1]:
import qiskit

from PatchedMeasCal.edge_bfs import CouplingMapGraph
from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.fake_backends import Grid, Hexagonal16, FullyConnected, SquareOctagonal


from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter

import qiskit.tools.jupyter

In [28]:
dev = SquareOctagonal(1, 2)
dev

<SquareOctagonal('Oct')>

In [29]:
patches = CouplingMapGraph(dev.configuration().coupling_map)
patches.edge_patches()

[[[0, 4], [12, 13], [10, 14], [7, 11]],
 [[0, 1], [2, 6], [9, 13]],
 [[1, 5], [2, 3], [11, 15]],
 [[3, 7], [6, 10]],
 [[4, 8]],
 [[5, 9], [14, 15], [8, 12]],
 [[5, 6]],
 [[9, 10]]]

In [2]:
def plus_state_prep(backend):
    n_qubits = backend.n_qubits
    coupling_map = backend._coupling_map
    
    targeted_qubits = [0]
    circuit = qiskit.QuantumCircuit(n_qubits, n_qubits)
    circuit.h(0)
    
    while len(targeted_qubits) < n_qubits:
        for i in range(n_qubits):
            if i not in targeted_qubits: # Qubit not yet added
                for edge in coupling_map: # For each edge in the map
                    if i in edge:
                        ctrl = edge[edge[0] == i]
                        targ = edge[edge[1] == i]
                        circuit.cnot(ctrl, targ)
                        targeted_qubits.append(i)
                        break
    circuit.measure(list(range(n_qubits)), list(range(n_qubits)))
    return circuit  

In [3]:
Grid(4, 4)

<Grid('square')>

In [4]:
n_qubits = 16
backends = [Grid(4, 4)]#, Hexagonal16()]#, FullyConnected(n_qubits)]

n_shots = 16000

fitters = [TensorPatchFitter(backend, n_shots=n_shots) for backend in backends]
[fitter.build(verbose=True) for fitter in fitters]

Building Coupling Graph
Building Edge Calibrations
	Building Calibration Circuits
	Building Edge Patches : [==================> ] 100.0% 
	Building Edge Patches : [===================>] 100.0% 
	Constructing Calibration Circuits : [================>   ] 100.0% 
	Executing Calibration Circuits
	De-hexing Measurement Results
	Tracing Patched Calibration Results : [================>   ] 100.0% 
Building Patch Calibrations
	Merging Patches : [=================>  ] 100.0% 
Building Measure Fitter
	Building Meas Fitters from Patches : [==================> ] 100.0% 


[None]

In [5]:
initial_layout = list(range(n_qubits))

for backend in backends[:1]:
    circ = plus_state_prep(backend)
    tc = qiskit.transpile(circ, backend=backend, initial_layout=initial_layout, optimization_level=0)
    results = qiskit.execute(tc, backend, shots=n_shots, initial_layout=initial_layout, optimization_level=0).result()

In [17]:
results.get_counts()['1' * 16]

2872

In [7]:
n_shots

16000

In [8]:
f_res = fitters[0].apply(results.get_counts())

In [18]:
f_res['1' * 16]

6283.463488538778